In [ ]:
try:
    import fitz  # PyMuPDF
except ImportError:
    !pip install pymupdf
    import fitz

In [ ]:
def parse_bookmarks_from_md(txt_path, offset):
    """
    从Markdown格式的文本文件中解析书签信息。
    
    该函数读取一个Markdown文件，其中书签以多级标题的形式呈现。每个标题后面紧跟着一个"@"符号和页码。
    根据标题前的井号数量来确定书签的层级。

    Args:
        txt_path (str): 需要解析的Markdown文件的路径。
        offset (int): 页码偏移量。对每个解析出来的页码值会增加这个偏移量。

    Returns:
        list: 一个列表，每个元素是另一个列表，包含层级、标题和页码（已加上偏移量）。
    """
    bookmarks = []  # 用于存储解析出来的书签信息
    with open(txt_path, 'r', encoding='utf-8') as file:  # 以只读模式打开文件
        for line in file:  # 遍历文件中的每一行
            stripped_line = line.strip()  # 去掉行首行尾的空白字符
            if stripped_line:  # 如果处理后的行不为空，则继续处理
                # 计算行开始的井号(#)数量来确定层级
                level = stripped_line.count('#', 0, stripped_line.find(' '))  # 井号后的第一个空格之前的井号数量为层级
                if level > 0:  # 如果存在井号，则认为是一个标题行
                    # 移除井号和空格，然后分割标题和页码
                    title_page_part = stripped_line[level:].strip()  # 移除层级标识符和前置空格
                    title, page = title_page_part.split('@')  # 按"@"符号分割标题和页码
                    bookmarks.append([level, title.strip(), int(page) + offset])  # 将解析出的信息添加到列表中
    return bookmarks  # 返回解析出的书签信息列表


In [ ]:
def add_bookmarks_to_pdf(pdf_path, output_pdf_path, bookmarks):
    """
    将书签信息添加到指定的PDF文件中，并保存到新的文件路径。

    Args:
        pdf_path (str): 源PDF文件的路径。
        output_pdf_path (str): 添加了书签后的PDF文件保存路径。
        bookmarks (list): 书签信息列表，每个书签是一个包含层级、标题和页码的列表。
                          例如：[[1, 'Chapter 1', 10], [2, 'Section 1.1', 12]]，
                          其中1和2表示书签的层级，'Chapter 1'和'Section 1.1'是标题，10和12是页码。
    """
    # 打开PDF文件
    doc = fitz.open(pdf_path)

    # 设置文档的目录（TOC）为提供的书签列表
    # PyMuPDF的set_toc方法接受一个列表，其中每个元素也是一个列表，表示一个书签。
    # 每个书签列表包含层级(level), 标题(title), 页码(page)。
    # 注意：页码从0开始计数，所以实际页码应该是书签中页码减一。
    bookmarks = [[level, title, page-1] for level, title, page in bookmarks]
    doc.set_toc(bookmarks)

    # 保存更改到新的PDF文件中，以防覆盖原文件
    doc.save(output_pdf_path)
    doc.close()  # 关闭文档
    print(f"书签已添加到新的PDF文件中：{output_pdf_path}")

In [ ]:
# 使用示例
bookmarks_md_path = "./input/Example.md"  # 书签Markdown文件路径
pdf_path = "./input/Example.pdf"  # 源PDF文件路径
output_pdf_path = "./output/Example.pdf"  # 输出PDF文件路径
offset = 0  # 如果你的书签页码需要调整（例如，PDF阅读器显示的页码与实际页码有偏差），设置这个偏移量

# 解析书签
bookmarks = parse_bookmarks_from_md(bookmarks_md_path, offset)

# 添加书签到PDF
add_bookmarks_to_pdf(pdf_path, output_pdf_path, bookmarks)